<a href="https://colab.research.google.com/github/Beta628427/Tesis/blob/master/2_Beta___Cod/Beta_PCA/PCA_Cod_master_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CODIGO MASTER PCA _ BETA

## En escodigo encontraremos PCA aplicado a los datos mmc3

__ Versión 1.0 __

In [289]:
pip install mglearn

In [0]:
import gspread, sklearn, mglearn, time, pandas as pd, numpy as np, scipy as sc
import matplotlib.pyplot as plt
from oauth2client.client import GoogleCredentials
from google.colab import auth
from sklearn.decomposition import PCA
from IPython.display import clear_output

# Autoriza Drive_BETA
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

## Abro <mmc3> y me ubico en "Joint Rotations"

In [0]:
sh            = gc.open('mmc3')                   # Abro documento 
worksheet     = sh.worksheet("Joint Rotations")   # Entro a la pestaña de Excel
rows          = worksheet.get_all_values()        # Saco todos los valores
df            = pd.DataFrame.from_records(rows)   # Guardo en una tabla panda

In [292]:
pd.DataFrame(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
0,Angle [°],% Gait Cycle,Natural,,,XS,,,S,,,M,,,L,,,Toe,,,Heel,,,Ascending,,,Descending,,,Natural,,,XS,,,S,,,M,,,L,,,Toe,,,Heel,,,Ascending,,,Descending,,
1,Pelvic Ant/Posterior Tilt,0,-7.28,-2.8,1.6,-7.2,-3.7,-0.2,-7.1,-2.6,1.9,-8.2,-3.6,1.0,-9.4,-4.4,0.5,-10.9,-6.5,-2.1,-12.4,-7.9,-3.4,-8.1,-3.3,1.6,-7.0,-2.4,2.2,-16.4,-9.3,-2.3,-15.7,-9.4,-3.1,-16.9,-9.1,-1.2,-14.6,-8.8,-3.0,-17.3,-12.8,-8.3,-19.0,-12.9,-6.7,-21.4,-14.7,-8.0,-18.0,-11.7,-5.4,-15.9,-9.2,-2.5
2,Pelvic Ant/Posterior Tilt,1,-7.2,-2.8,1.6,-7.2,-3.7,-0.3,-7.0,-2.6,1.8,-8.2,-3.6,1.0,-9.2,-4.4,0.5,-10.8,-6.4,-2.1,-12.3,-7.8,-3.3,-8.5,-3.7,1.1,-7.1,-2.6,1.9,-16.4,-9.4,-2.4,-15.9,-9.5,-3.2,-16.9,-9.1,-1.2,-14.7,-8.9,-3.0,-17.3,-12.8,-8.3,-19.0,-12.8,-6.7,-21.3,-14.6,-8.0,-18.3,-12.0,-5.7,-15.9,-9.2,-2.5
3,Pelvic Ant/Posterior Tilt,2,-7.2,-2.8,1.6,-7.2,-3.8,-0.4,-7.0,-2.6,1.8,-8.2,-3.5,1.1,-9.2,-4.3,0.5,-10.6,-6.3,-2.1,-12.2,-7.7,-3.2,-8.9,-4.1,0.6,-7.3,-2.8,1.7,-16.5,-9.5,-2.5,-16.1,-9.7,-3.3,-17.0,-9.2,-1.3,-14.8,-9.0,-3.2,-17.3,-12.9,-8.5,-18.9,-12.8,-6.8,-21.3,-14.7,-8.1,-18.6,-12.3,-6.0,-16.0,-9.3,-2.6
4,Pelvic Ant/Posterior Tilt,3,-7.2,-2.7,1.7,-7.2,-3.8,-0.4,-7.0,-2.6,1.8,-8.1,-3.5,1.2,-9.1,-4.3,0.6,-10.5,-6.3,-2.0,-12.2,-7.7,-3.2,-9.3,-4.6,0.0,-7.4,-2.9,1.6,-16.7,-9.6,-2.6,-16.3,-9.8,-3.4,-17.1,-9.2,-1.4,-15.0,-9.1,-3.3,-17.4,-13.1,-8.8,-18.8,-12.9,-6.9,-21.4,-14.8,-8.3,-19.0,-12.7,-6.4,-16.1,-9.4,-2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,Foot Int/External Rotation,96,-21.1,-13.4,-5.7,-18.6,-12.5,-6.3,-22.7,-14.1,-5.4,-19.8,-12.4,-5.0,-20.8,-12.2,-3.5,-13.6,-7.0,-0.4,-22.7,-14.7,-6.8,-21.0,-12.9,-4.8,-24.4,-12.4,-0.5,-19.8,-12.9,-6.0,-18.9,-12.3,-5.6,-20.2,-13.5,-6.9,-19.7,-13.1,-6.5,-17.2,-11.9,-6.5,-9.8,-4.0,1.9,-20.8,-14.7,-8.6,-21.4,-14.4,-7.3,-20.1,-13.2,-6.4
1411,Foot Int/External Rotation,97,-20.7,-13.1,-5.6,-18.5,-12.3,-6.1,-22.3,-13.9,-5.5,-19.4,-12.1,-4.9,-20.2,-11.8,-3.3,-13.5,-6.8,-0.2,-22.6,-14.6,-6.5,-20.1,-11.6,-3.1,-23.9,-11.6,0.6,-19.4,-12.6,-5.8,-18.9,-12.2,-5.4,-19.9,-13.2,-6.5,-19.1,-12.8,-6.4,-16.7,-11.3,-5.8,-9.4,-3.7,2.1,-20.5,-14.5,-8.4,-20.3,-13.2,-6.1,-19.5,-12.7,-5.9
1412,Foot Int/External Rotation,98,-20.1,-12.9,-5.7,-18.3,-12.0,-5.8,-21.6,-13.6,-5.5,-18.9,-11.9,-4.9,-19.6,-11.4,-3.2,-13.6,-6.9,-0.1,-22.4,-14.3,-6.2,-19.4,-10.6,-1.8,-23.5,-11.0,1.6,-19.0,-12.3,-5.6,-18.8,-12.0,-5.2,-19.4,-12.8,-6.2,-18.5,-12.4,-6.2,-16.4,-10.7,-4.9,-9.0,-3.4,2.3,-20.2,-14.1,-8.1,-19.4,-12.2,-5.0,-19.0,-12.2,-5.4
1413,Foot Int/External Rotation,99,-19.4,-12.5,-5.6,-18.0,-11.6,-5.3,-21.0,-13.1,-5.3,-18.3,-11.6,-4.9,-18.9,-11.0,-3.2,-13.9,-7.0,-0.1,-22.2,-14.1,-6.0,-18.9,-9.8,-0.7,-23.3,-10.5,2.3,-18.4,-11.8,-5.3,-18.5,-11.7,-4.9,-18.8,-12.3,-5.8,-17.8,-11.9,-6.0,-16.1,-10.1,-4.1,-8.7,-3.2,2.4,-19.7,-13.8,-7.9,-18.9,-11.5,-4.2,-18.6,-11.9,-5.1


## Función de realizar numeros aleatorios con una media y una desviacion estandar especifica


In [0]:
# PROCESO DE GENERAR NUMEROS ALEATORIOS CON PROMEDIO Y DESVIACION ESTANDAR 
# DESEADOS
#···············································································
num_samples     = 20     # Numero de numeros aleatorios a generar
desired_mean    = -2.8   # promedio de numeros aleatorios
desired_std_dev = 8.9    # Desviacion estandar a numeros aleatorios a generar
#···············································································
# Funcion de variables aleatorias con media y desviacion estandar
def var_alea (num_samples,desired_mean,desired_std_dev):
  samples = np.random.normal(loc=0.0, scale=desired_std_dev, size=num_samples)
  actual_mean = np.mean(samples)
  actual_std = np.std(samples)
  #print("Initial samples stats   : mean = {:.4f} stdv = {:.4f}".
  #      format(actual_mean, actual_std))
  zero_mean_samples = samples - (actual_mean)
  zero_mean_mean = np.mean(zero_mean_samples)
  zero_mean_std = np.std(zero_mean_samples)
  #print("True zero samples stats : mean = {:.4f} stdv = {:.4f}".
  #      format(zero_mean_mean, zero_mean_std))
  scaled_samples = zero_mean_samples * (desired_std_dev/zero_mean_std)
  scaled_mean = np.mean(scaled_samples)
  scaled_std = np.std(scaled_samples)
  #print("Scaled samples stats    : mean = {:.4f} stdv = {:.4f}".
  #      format(scaled_mean, scaled_std))
  final_samples = scaled_samples + desired_mean
  final_mean = np.mean(final_samples)
  final_std = np.std(final_samples)
  #print("Final samples stats     : mean = {:.4f} stdv = {:.4f}".
  #      format(final_mean, final_std))
  #print(final_samples)
  return(final_samples)

### ¡Aquí! vuelvo una tabla en 20 tablas que almacena las variables tanto de jovenes como las de viejos de los targets < "Natural","XS","S","M","L","Toe","Heel","Ascending","Descending" > con media y desviacion dada:

#### 1) la media es el valor medio de cada target
#### 2) la desviacion estandar es la suma de los valores absolutos de la desviacion estandar positima mas la negativa.

#### Nota····································································
#### solo salieron 25 lineas de codigo XD solo que para que se vea mas bonito consume mas lineas 

\\(·w·)/


In [0]:
# Seleccionando de la sheet "Joint Rotations"
# ······· Pelvic_Ant_Posterior_Tilt
f_1 = 1   # fila de inicio Pelvic_Ant_Posterior_Tilt
f_2 = 102 # final final    Pelvic_Ant_Posterior_Tilt
targets         = df.iloc[0,:]
targets         = pd.DataFrame(targets[np.newaxis,:])
ta              = pd.DataFrame(np.array(df.iloc[f_1:f_2,:]))
tabla           = pd.DataFrame(ta)
target_colum = ['Angle [°]', '% Gait Cycle', 
                'joven_Natural-',   'joven_Natural',   'joven_Natural+', 
                'joven_XS-',        'joven_XS',        'joven_XS+', 
                'joven_S-',         'joven_S',         'joven_S+', 
                'joven_M-',         'joven_M',         'joven_M+', 
                'joven_L-',         'joven_L',         'joven_L+', 
                'joven_Toe-',       'joven_Toe',       'joven_Toe+', 
                'joven_Heel-',      'joven_Heel',      'joven_Heel+', 
                'joven_Ascending-', 'joven_Ascending', 'joven_Ascending+', 
                'joven_Descending-','joven_Descending','joven_Descending+',
                'viejo_Natural-',   'viejo_Natural',   'viejo_Natural+', 
                'viejo_XS-',        'viejo_XS',        'viejo_XS+', 
                'viejo_S-',         'viejo_S',         'viejo_S+', 
                'viejo_M-',         'viejo_M',         'viejo_M+', 
                'viejo_L-',         'viejo_L',         'viejo_L+', 
                'viejo_Toe-',       'viejo_Toe',       'viejo_Toe+', 
                'viejo_Heel-',      'viejo_Heel',      'viejo_Heel+', 
                'viejo_Ascending-', 'viejo_Ascending', 'viejo_Ascending+', 
                'viejo_Descending-','viejo_Descending','viejo_Descending+']
tabla.columns = target_colum # Agregarle targets a las columnas de tabla
#···············································································
# Variables diccionario de las tablas de 20 personas para jovenes y adultas
per = [""   ,"p1" ,"p2" ,"p3" ,"p4" ,"p5" ,"p6" ,"p7" ,"p8" ,"p9" ,"p10",
       "p11","p12","p13","p14","p15","p16","p17","p18","p19","p20"]
personas= {per[1]:[] ,per[2]:[] ,per[3]:[] ,per[4]:[] ,per[5] :[],
           per[6]:[] ,per[7]:[] ,per[8]:[] ,per[9]:[] ,per[10]:[],
           per[11]:[],per[12]:[],per[13]:[],per[14]:[],per[15]:[],
           per[16]:[],per[17]:[],per[18]:[],per[19]:[],per[20]:[]}
target_colum_ = ['% Gait Cycle','joven_Natural','joven_XS',
                 'joven_S','joven_M','joven_L','joven_Toe','joven_Heel', 
                 'joven_Ascending','joven_Descending','viejo_Natural',
                 'viejo_XS','viejo_S','viejo_M','viejo_L','viejo_Toe',
                 'viejo_Heel','viejo_Ascending','viejo_Descending']
for i in range(1,21):
  personas[per[i]] = pd.DataFrame(np.zeros((101,19)))
  personas[per[i]].columns = target_colum_
#···············································································
# Lleno las distinatas tablas de personas
j_v = ["joven","viejo"]
tag = ["Natural","XS","S","M","L","Toe","Heel","Ascending","Descending"]
# Armar el tag
for _j_v in j_v:
  for _tag in tag:
    vari = _j_v+"_"+_tag
    for i in range(tabla[vari].shape[0]):
      mean = tabla[vari][i]
      _mas = tabla[vari+"+"][i]
      _men = tabla[vari+"-"][i]
      DE   = abs(float(_mas))+abs(float(_men))
      NP   = pd.DataFrame(per).shape[0]-1 # numero de personas
      num_gener = var_alea(NP,float(mean),DE) # genero los numeros aleatorios
      # Comprovamos valores demedia y desviacion estandar
      media = np.mean(num_gener)
      DEE = np.std(num_gener)
      # Ploteo info
      #print("··············································· "+
      #      "i = "+str(i)+" / "+str(vari)+
      #      " ···············································"+"\n"+
      #      "media = "+str(mean)+" / DE+ = "+str(_mas)+" / DE- = "+str(_men)+
      #      " / DE total = "+str(DE)+"\n"+
      #      "···············································"+"\n"+
      #      "media = "+str(media)+" / DE total = "+str(DEE)+"\n"+
      #      "···············································"+
      #      "\n"+str(num_gener))
      # Lleno datos a las diferentes tablas de las 20 personas
      for _per in range(1,pd.DataFrame(per).shape[0]):
        #print("vari = "+str(vari))
        #print("Lista la persona = "+per[_per])
        personas[per[_per]][vari][i] = num_gener[_per-1]

### Comprobacion de valores
#### Replicaremos la tabla de mmc3 con el fin de verificar que sea los mismos valores de promedio y desviacion estandar positivo como negativo 


In [295]:
# Hallamos la media de un valor con coordenadas (n,m) para todas las personas
valores = []
for _per in range(1,pd.DataFrame(per).shape[0]):
  valores.append(personas[per[_per]]['joven_Natural'][0]) 
  print(str(personas[per[_per]][target_colum_[1]][0])+"    "+str(media))
media = np.mean(valores)
DEE   = np.std(valores)
print("\nmedia = " + str(media))
print("DEE = " + str(DEE))

-15.21126886837424    -10.000000000000002
6.960652586844906    -10.000000000000002
-6.673387632233789    -10.000000000000002
7.473802967505187    -10.000000000000002
3.722994074743716    -10.000000000000002
-9.803186342455524    -10.000000000000002
-7.2010937469612575    -10.000000000000002
10.679395468804952    -10.000000000000002
-11.058660576704682    -10.000000000000002
-0.4076048103924239    -10.000000000000002
-17.33580417617096    -10.000000000000002
-3.6933425652131375    -10.000000000000002
1.2404496133662128    -10.000000000000002
-15.137514047965674    -10.000000000000002
-0.8675871370111026    -10.000000000000002
-5.408959706553635    -10.000000000000002
-10.568547632369757    -10.000000000000002
-3.7155155330553287    -10.000000000000002
14.647633519964913    -10.000000000000002
6.357544544231627    -10.000000000000002

media = -2.7999999999999994
DEE = 8.88


### PLOTEO EL DICCIONARIO DE PERSONAS <Pelvic Ant/Posterior Tilt>:
#### 1) Este diccionario tiene almacenadas los datos de las 20 personas
#### 2) Por personas hay datos de jovenes y adultos de los difernetes targets

In [296]:
personas

{'p1':      % Gait Cycle  joven_Natural  ...  viejo_Ascending  viejo_Descending
 0             0.0     -15.211269  ...        -4.172170        -14.109177
 1             0.0       2.313344  ...        -5.567475          2.797415
 2             0.0      -2.641185  ...       -26.552801        -33.325439
 3             0.0       3.591847  ...         7.040981          1.060896
 4             0.0      -0.567683  ...        19.468945         23.513914
 ..            ...            ...  ...              ...               ...
 96            0.0      19.134815  ...         7.910411         -9.713778
 97            0.0      -1.190304  ...         1.660645          9.578823
 98            0.0      -7.570382  ...        25.741894          4.931738
 99            0.0       7.678209  ...       -33.578939        -29.527239
 100           0.0       4.994829  ...       -17.628697        -15.854413
 
 [101 rows x 19 columns],
 'p10':      % Gait Cycle  joven_Natural  ...  viejo_Ascending  viejo_Descendi

In [297]:
pd.DataFrame(personas["p1"])

,% Gait Cycle,joven_Natural,joven_XS,joven_S,joven_M,joven_L,joven_Toe,joven_Heel,joven_Ascending,joven_Descending,viejo_Natural,viejo_XS,viejo_S,viejo_M,viejo_L,viejo_Toe,viejo_Heel,viejo_Ascending,viejo_Descending
0,0.0,-15.211269,3.993686,-16.916269,-9.844078,3.490601,-2.094754,4.787279,-15.125198,-9.230672,-40.501144,-7.161049,-30.441484,-10.669264,-37.478299,0.409291,-22.928684,-4.172170,-14.109177
1,0.0,2.313344,-0.721956,-13.833086,-24.028710,5.793009,-5.907314,-27.521396,-14.985456,-3.930588,-12.971676,-13.073573,-0.562947,-10.575451,-44.061865,-22.427198,-20.695959,-5.567475,2.797415
2,0.0,-2.641185,-4.408309,-5.798753,-4.818301,6.769722,-2.217716,-1.844360,-4.936533,7.331886,-32.359286,-22.720901,-4.523642,4.602568,-3.202069,-37.780334,17.231335,-26.552801,-33.325439
3,0.0,3.591847,-5.397210,-0.119622,-1.852312,-9.411759,-1.842862,-21.187465,4.059698,6.469722,-40.337530,-4.649726,16.045266,-20.266316,-21.308955,12.147617,-55.646011,7.040981,1.060896
4,0.0,-0.567683,-5.070818,-12.651312,3.631622,-8.044399,-1.495113,-31.125832,-15.459518,14.312989,-8.660151,-12.274610,-33.201039,0.431389,26.054446,-43.105383,-24.579555,19.468945,23.513914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,19.134815,-5.611695,6.214077,2.099836,-12.532734,-24.088914,-0.701804,-14.952672,1.195405,0.961640,3.852878,-3.509806,-22.870352,-41.577769,-13.875120,-69.433061,7.910411,-9.713778
97,0.0,-1.190304,-3.578521,-9.052087,-6.089197,11.353827,-29.326121,5.399874,-37.875663,-12.892472,1.062466,-18.637078,22.102437,-28.073604,-36.190366,-21.352770,16.891779,1.660645,9.578823
98,0.0,-7.570382,-12.507534,-0.262834,15.646852,-9.948943,-28.363267,17.482816,-15.579294,-8.276609,-11.227004,17.862087,-24.170471,-12.909394,-19.082604,-36.551175,18.262901,25.741894,4.931738
99,0.0,7.678209,1.887674,12.044118,-5.065227,0.714405,-4.807002,17.427312,-12.252538,3.446455,12.119120,-19.955654,-3.278922,-28.590432,-35.778864,-11.777166,16.600265,-33.578939,-29.527239


In [298]:
pd.DataFrame(personas["p10"])

,% Gait Cycle,joven_Natural,joven_XS,joven_S,joven_M,joven_L,joven_Toe,joven_Heel,joven_Ascending,joven_Descending,viejo_Natural,viejo_XS,viejo_S,viejo_M,viejo_L,viejo_Toe,viejo_Heel,viejo_Ascending,viejo_Descending
0,0.0,-0.407605,-9.886224,-0.073324,-0.248124,-1.219530,-1.058859,-7.879294,6.450526,-10.879664,-18.216090,-38.472185,-31.784832,-5.506580,-42.119627,1.663676,-53.759401,28.469873,2.077552
1,0.0,0.328432,-13.139221,10.307209,-12.737045,-6.361004,-9.082322,-33.741311,5.218331,-13.697220,-18.291245,7.694078,5.833767,-11.977729,29.676235,2.574680,-23.959268,21.530419,9.741164
2,0.0,4.401967,-5.771947,2.227834,-6.837218,-17.057267,-17.736040,1.887266,-6.212420,4.922885,-11.297261,-31.117244,-18.603383,5.894519,30.518883,-16.912741,-3.405172,-30.290086,-13.134215
3,0.0,-5.379863,-17.533492,-2.487554,1.182400,-14.064534,21.280240,-6.725648,-9.640974,3.255947,-18.997630,-1.362873,-23.906707,-13.392725,-20.194018,-29.983478,-11.942558,-48.688768,-15.214883
4,0.0,10.297578,-3.262150,6.650730,-10.045485,-23.924240,2.377235,-37.814271,9.327857,-8.835688,20.968352,-48.334774,-0.851048,8.809394,-51.580790,-13.912496,5.840435,-71.157699,-14.104101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,2.047564,-7.932872,-6.128242,-6.238495,13.677296,-1.791216,-15.178641,-26.636931,-1.962999,10.323026,-23.829396,-25.059061,-8.389776,8.469122,-6.855797,6.986907,-2.328444,-2.425626
97,0.0,-10.219596,-16.471062,5.865660,-21.798292,19.643527,3.070815,-10.289053,-13.479795,35.137273,7.471512,-63.745579,-1.192576,-21.051486,14.738298,-49.954672,-38.627906,-52.501002,-8.065008
98,0.0,6.127483,5.612366,-3.241610,-12.291533,-19.418237,-12.788110,2.799406,-14.425306,22.829545,-0.278436,-6.788985,-4.207922,-16.693408,-16.835247,-20.184097,-25.920764,-44.923986,-26.833066
99,0.0,-0.327398,-1.969140,-7.438762,-14.662357,-24.170220,9.384736,-3.365747,-3.041009,-12.037108,-8.768733,12.672288,10.989235,-22.761631,-26.429755,12.753184,-15.971708,-9.709906,-8.385072


In [299]:
pd.DataFrame(personas["p20"])

,% Gait Cycle,joven_Natural,joven_XS,joven_S,joven_M,joven_L,joven_Toe,joven_Heel,joven_Ascending,joven_Descending,viejo_Natural,viejo_XS,viejo_S,viejo_M,viejo_L,viejo_Toe,viejo_Heel,viejo_Ascending,viejo_Descending
0,0.0,6.357545,-9.876875,2.695964,-5.221061,9.384693,7.363149,-12.315014,2.696290,1.823722,-8.847518,-8.303564,-38.929055,-6.282469,-48.551567,-7.432420,-26.264606,-3.309878,-28.306060
1,0.0,2.386596,-13.315339,9.931102,-8.556889,4.704306,-4.124955,15.586166,-6.747513,-7.142133,5.351446,-19.136961,21.521154,-17.195336,50.056860,-6.159173,-38.885615,25.365656,-19.075695
2,0.0,-11.132509,-7.561890,-1.447737,-11.025559,5.913868,-0.691354,-20.921536,-5.602425,-7.123418,-10.646951,7.525967,13.115048,3.785940,-4.653302,-1.425946,0.408558,-35.735211,-50.255601
3,0.0,4.445065,-1.435533,16.014517,-4.807939,-12.659476,10.939921,-24.108059,-7.055640,-11.840027,17.445988,-11.118073,-2.882559,-20.763978,13.933845,-13.580908,0.034869,38.904389,-42.893478
4,0.0,3.413135,-7.035110,8.522007,-10.407815,0.141108,-14.473600,-16.750074,9.544490,-0.389241,-34.065052,-6.493905,12.826343,-17.076218,10.146687,-3.270564,-71.247307,-25.282013,4.802397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,-11.349536,-13.311665,16.147852,-1.299488,8.324948,-24.209472,-38.998217,-5.084999,-25.515795,-14.414235,0.803899,-2.467387,-15.761428,-9.401237,6.707324,7.477177,-36.217490,-29.823845
97,0.0,-7.601850,7.711399,-5.636559,-2.771317,-9.159843,-5.636258,-25.541897,6.575936,-4.492120,-14.211554,-16.841252,-18.079998,-24.950751,-6.113615,4.104368,-50.155950,-30.057221,-17.056706
98,0.0,-8.276418,3.466964,-0.544784,-5.242953,-8.896281,3.586955,-12.989832,-30.513907,-20.947446,-2.775050,-13.548556,21.949814,-8.021095,-21.372838,17.191783,-25.576602,10.230178,5.211236
99,0.0,-7.099746,-11.940531,4.804742,-12.567641,6.435487,-14.384379,18.747830,-17.311487,-17.562547,1.146460,-6.978233,-35.677707,-29.885407,-4.342516,-72.374762,-7.139742,-49.924333,-62.820337


In [300]:
pd.DataFrame(personas["p1"]["joven_XS"])

,joven_XS
0,3.993686
1,-0.721956
2,-4.408309
3,-5.397210
4,-5.070818
...,...
96,-5.611695
97,-3.578521
98,-12.507534
99,1.887674


In [301]:
pd.DataFrame(personas["p1"]["viejo_XS"][0:10])

,viejo_XS
0,-7.161049
1,-13.073573
2,-22.720901
3,-4.649726
4,-12.274610
5,-47.945890
6,-10.033664
7,-46.314139
8,-1.956600
9,20.053755


  ### Guardamos la info en un Google Sheet de las diferentes personas de las pruebas

In [0]:
# Creamos un documento Google sheet para poner la informacion de las 20 personas
# OJO! Solo correr una vez
gc.create("mmc3_personas")
sh2 = gc.open('mmc3_personas')
for i in range(1,21):
  sh2.add_worksheet(title=per[i],rows='110',cols='25')
sh2.add_worksheet(title='MMC3',rows='1450',cols='60')

In [0]:
# Examina información de la persona a su target
personas[per[1]][target_colum_[1]][0]

In [0]:
wks    = sh2.worksheet(per[1])
wks.update_acell('A1',"beta")

In [0]:
abc = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R',
       'S','T','U','V','W','X','Y','Z',
       'AA','AB','AC','AD','AE','AF','AG','AH','AI','AJ','AK','AL','AM','AN',
       'AO','AP','AQ','AR','AS','AT','AU','AV','AW','AX','AY','AZ',
       'BA','BB','BC','BD','BE','BF','BG','BH','BI','BJ','BK','BL','BM','BN',
       'BO','BP','BQ','BR','BS','BT','BU','BV','BW','BX','BY','BZ']
#···············································································
# en el Google sheet mmc3_personas
# Actualizo los targets de columnas en Google sheet por cada pagina de personas
for i in range(1,pd.DataFrame(per).shape[0]):
  wks    = sh2.worksheet(per[i])
  for j in range(pd.DataFrame(target_colum_).shape[0]):
      wks.update_acell(abc[j]+'1',target_colum_[j])
# Actualizo los targets del sheet "MMC3" con los mismos target de columnas que
# el mmc3
wks    = sh2.worksheet("MMC3")
for i in range(pd.DataFrame(target_colum).shape[0]):
  #print("======== "+str(i))
  #print(target_colum[i])
  #print(abc[i])
  wks.update_acell(abc[i]+'1',target_colum[i])
#···············································································
# Lleno todas las tablas del Google sheet mmc3_personas



## MODIFICAR LOS PCA POR QUE SE CAMBIO EL DATASET

In [320]:
# Filas velocidades de personas 

for i in range(1,pd.DataFrame(per).shape[0]):

  # xx1 = np.array(range(0,101))[:, np.newaxis]    # 0 a 100 ['% Gait Cycle']
  xx2 = np.array(personas[per[i]].iloc[:,1 :10]) # jovenes 1:10
  xx3 = np.array(personas[per[i]].iloc[:,10:19]) # viejos  10:19

  a = np.array(pd.DataFrame(xx2).T) # joven
  b = np.array(pd.DataFrame(xx3).T) # viejo

  aa = np.concatenate((aa,a),axis=0) # Concateno jovenes
  bb = np.concatenate((bb,b),axis=0) # Concateno viejos

  cc = np.concatenate((aa,bb), axis=0)

# Columnas Articulaciones 0 -100 % < % Gait Cycle' >

pd.DataFrame(cc)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-15.211269,2.313344,-2.641185,3.591847,-0.567683,-0.070975,-1.659356,-4.609746,3.893081,-16.305264,0.234760,-0.316875,-2.207499,4.702006,-4.546793,18.250508,-8.619646,6.091428,11.415568,-5.190493,1.299573,8.191199,7.324961,-9.747982,-10.041649,-4.337735,6.245958,8.575465,-8.096297,3.003575,1.970108,2.026379,-17.406264,1.868234,-1.792276,-13.490204,-4.478083,6.427120,-16.971469,-14.058382,...,-5.310023,-12.858553,16.619880,0.055475,9.621790,8.466024,0.861706,5.512611,4.201301,2.650819,2.530265,-13.616645,12.544105,-5.844714,-3.451778,-13.864389,-14.697601,-3.542888,-24.093122,0.003939,-2.870825,-19.474253,-2.064343,1.591573,-9.468323,-3.249262,-0.996223,-10.451737,0.068099,-14.145441,2.516767,-10.548908,0.197544,2.548929,-16.315397,19.134815,-1.190304,-7.570382,7.678209,4.994829
2,3.993686,-0.721956,-4.408309,-5.397210,-5.070818,-3.817329,-4.396696,4.853063,5.648356,-0.180348,-1.558713,-8.243164,-13.098764,-8.787275,-3.277612,-4.070212,3.861568,-8.620306,-13.061043,-3.011238,1.703046,2.356916,2.091922,-6.795321,-15.672674,-17.341186,0.340589,-6.973513,1.572324,-12.933194,-2.032424,-6.424384,-11.690834,-7.613478,8.900306,-4.042699,-2.553128,-8.321790,-9.385370,6.486704,...,-16.669618,-1.932273,-6.282123,4.817373,-7.197177,-9.038882,-4.649353,2.974739,-8.893870,-4.391195,-0.400906,-1.530329,-0.847350,-2.383661,4.600085,-11.957826,16.004964,9.269761,0.560277,8.401578,7.334858,3.181898,-2.632000,-18.590400,0.346553,3.355324,-1.437693,14.832053,-15.013887,6.229704,-14.098185,0.275572,-5.633557,1.718400,-3.597376,-5.611695,-3.578521,-12.507534,1.887674,3.685504
3,-16.916269,-13.833086,-5.798753,-0.119622,-12.651312,-12.729717,5.534728,-6.107563,-5.127834,-3.737522,-5.776074,-6.573648,-1.136090,-10.604615,-2.630031,10.942822,-3.312634,-7.855251,9.088668,2.856398,-2.951485,-8.878707,-25.009281,-2.895405,-4.537884,-9.967728,-7.566626,7.104628,5.626697,15.256461,-8.698084,-0.849258,-8.194661,-1.235164,2.150077,-6.127206,-9.482762,-6.850850,-16.022650,12.656965,...,12.242851,-14.230662,5.400344,-4.570329,-16.141705,1.975190,-16.947114,-3.413001,-20.314938,-2.973971,-7.463825,-0.810689,-5.679880,-4.361406,19.541238,2.369578,8.985101,-12.632510,-11.409791,2.263269,3.725993,-19.885131,9.726152,-6.622545,-10.573671,-14.254660,-2.008721,-7.144853,-7.974685,8.949731,4.420201,3.378570,-2.311531,-1.576263,2.364115,6.214077,-9.052087,-0.262834,12.044118,-6.924980
4,-9.844078,-24.028710,-4.818301,-1.852312,3.631622,22.980558,7.802745,2.456858,-8.117605,0.576179,-19.295902,-4.675231,16.926954,10.324280,-6.319794,-14.468369,5.484441,-11.531945,8.373483,5.548762,-22.837436,4.655297,-2.535145,-15.677296,-6.920597,-18.005542,4.532278,3.049864,-13.565650,-7.820592,-11.435874,-19.954040,-3.055032,-16.589817,-20.731199,-13.793728,-10.479826,-2.815966,-19.914407,-9.406499,...,-12.484792,3.864452,-0.005747,-7.967086,-3.301893,-2.950228,-1.549676,12.824335,-19.545521,19.470469,-12.899522,5.537040,-5.993771,-15.404111,-13.501547,-11.567206,-12.747485,-14.363107,-5.383629,-11.638269,2.005090,-1.028638,16.396415,-4.1

In [0]:
['% Gait Cycle','joven_Natural','joven_XS',
'joven_S','joven_M','joven_L','joven_Toe','joven_Heel', 
'joven_Ascending','joven_Descending','viejo_Natural',
'viejo_XS','viejo_S','viejo_M','viejo_L','viejo_Toe',
'viejo_Heel','viejo_Ascending','viejo_Descending']

In [0]:
for i in range(1,pd.DataFrame(per).shape[0]):

  xx1 = np.array(range(0,101))[:, np.newaxis]    # 0 a 100 ['% Gait Cycle']
  xx2 = np.array(personas[per[i]].iloc[:,1 :10]) # jovenes 1:10
  xx3 = np.array(personas[per[i]].iloc[:,10:19]) # viejos  10:19

  X_jov = np.concatenate((xx1,np.array(xx2)),axis=1) # Concateno jovenes
  X_vie = np.concatenate((xx1,np.array(xx3)),axis=1) # Concateno viejos



In [0]:
targets                       = df.iloc[0,:]
targets                       = pd.DataFrame(targets[np.newaxis,:])
Pelvic_Ant_Posterior_Tilt     = np.concatenate((np.array(targets),np.array(df.iloc[1:102,:])),axis=0)
# creo X
fila_1 = 1
fila_2 = fila_1 + 101
# concateno de 0-100% con los valores q deseo graficar como coordenadas X
X1 = np.concatenate((np.array(df.iloc[fila_1:fila_2,[1]]),
                     np.array(df.iloc[fila_1:fila_2,
                                      [3,6,7,12,15,18,21,24,27]])),   
                    axis=1)
X2 = np.concatenate((np.array(df.iloc[fila_1:fila_2,[1]]),
                     np.array(df.iloc[fila_1:fila_2,
                                      [30,33,36,39,42,45,48,51,54]])),
                    axis=1)
pd.DataFrame(X1)
X  = np.concatenate((X1,X2),axis=0)
pd.DataFrame(X)
# creo Y
Y = []
for i in range(X1.shape[0]):
  Y.append(int(0))
for i in range(X2.shape[0]):
  Y.append(int(1))
#
Y = np.array(Y)
print(pd.DataFrame(X))
#
print(type(X))
print(type(Y))

In [0]:
X = np.array(X)
Y = Y[:, np.newaxis]

In [0]:
for i in range(1,10):
  plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, i], color='red',  marker='^', alpha=0.5) # jovenes
  plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, i], color='blue', marker='o', alpha=0.5) # viejos
  
plt.grid(True)
plt.title("G _ Pelvic Ant / Posterior Tilt")
plt.xlabel("Gait Cycle [%]")
plt.ylabel("Pelvic Ant / Posterior Tilt [deg]")
plt.yscale('linear')
plt.xscale('linear')
plt.tight_layout()
plt.show()

In [0]:
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 1], color='red',  marker='^', alpha=0.5) # jovenes
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 2], color='red',  marker='^', alpha=0.5) # jovenes
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 3], color='red',  marker='^', alpha=0.5) # jovenes
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 4], color='red',  marker='^', alpha=0.5) # jovenes
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 5], color='red',  marker='^', alpha=0.5) # jovenes
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 6], color='red',  marker='^', alpha=0.5) # jovenes
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 7], color='red',  marker='^', alpha=0.5) # jovenes
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 8], color='red',  marker='^', alpha=0.5) # jovenes
plt.scatter(X[Y[:,0] == 0, 0], X[Y[:,0] == 0, 9], color='red',  marker='^', alpha=0.5) # jovenes

plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 1], color='blue', marker='o', alpha=0.5) # viejos
plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 2], color='blue', marker='o', alpha=0.5) # viejos
plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 3], color='blue', marker='o', alpha=0.5) # viejos
plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 4], color='blue', marker='o', alpha=0.5) # viejos
plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 5], color='blue', marker='o', alpha=0.5) # viejos
plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 6], color='blue', marker='o', alpha=0.5) # viejos
plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 7], color='blue', marker='o', alpha=0.5) # viejos
plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 8], color='blue', marker='o', alpha=0.5) # viejos
plt.scatter(X[Y[:,0] == 1, 0], X[Y[:,0] == 1, 9], color='blue', marker='o', alpha=0.5) # viejos

plt.grid(True)
plt.title("G _ Pelvic Ant / Posterior Tilt")
plt.xlabel("Gait Cycle [%]")
plt.ylabel("Pelvic Ant / Posterior Tilt [deg]")
plt.yscale('linear')
plt.xscale('linear')
plt.tight_layout()
plt.show()

In [0]:
from sklearn.decomposition import PCA
#
pca=PCA(n_components=2)
pca.fit(X)
#
transformada=pca.transform(X)

In [0]:
transformada.shape

In [0]:
plt.scatter(transformada[Y[:,0] == 0,0],transformada[Y[:,0] == 0,1], color='red',  marker='^', alpha=0.5) 
plt.scatter(transformada[Y[:,0] == 1,0],transformada[Y[:,0] == 1,1], color='blue', marker='o', alpha=0.5) 
plt.grid(True)
plt.title("PCA _ Pelvic Ant / Posterior Tilt")
plt.xlabel("PCA_1")
plt.ylabel("PCA_2")
plt.yscale('linear')
plt.xscale('linear')
plt.tight_layout()
plt.show()